## 3.2 케라스를 이용한 구현

### Sequential 모델

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Sequential 모델을 사용하는 경우

`Sequential` 모델은 각 레이어에 **정확히 하나의 입력 텐서와 하나의 출력 텐서**가 있는 **일반 레이어 스택**에 적합합니다.

개략적으로 다음과 같은 `Sequential` 모델은

In [5]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"), #맨 앞 param이 뉴런수, w=(3,2), b=(2,)
        layers.Dense(3, activation="relu", name="layer2"), # w=(2,3), b=(3,)
        layers.Dense(4, name="layer3"),                    #출력층은 affine연산, w=(3,4), b=(4,)
    ]
)

x = tf.ones((3, 3))
y = model(x)
print(y.shape)
print(y)

(3, 4)
tf.Tensor(
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(3, 4), dtype=float32)


다음 함수와 동일합니다.

In [7]:
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))
print(y.shape)
print(y)

(3, 4)
tf.Tensor(
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(3, 4), dtype=float32)


In [10]:
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

x = tf.ones((3, 3))
x = layer1(x)
x = layer2(x)
y = layer3(x)
print(y.shape)
print(y)

(3, 4)
tf.Tensor(
[[-0.04867636 -0.03222826  0.15806258 -0.12221034]
 [-0.04867636 -0.03222826  0.15806258 -0.12221034]
 [-0.04867636 -0.03222826  0.15806258 -0.12221034]], shape=(3, 4), dtype=float32)


Sequential 모델은 다음의 경우에 **적합하지 않습니다**.

- 모델에 다중 입력 또는 다중 출력이 있습니다
- 레이어에 다중 입력 또는 다중 출력이 있습니다
- 레이어 공유를 해야 합니다
- 비선형 토폴로지를 원합니다(예: 잔류 연결, 다중 분기 모델)

### Sequential 모델 생성하기

레이어의 목록을 Sequential 생성자에 전달하여 Sequential 모델을 만들 수 있습니다.

In [11]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

속한 레이어는 `layers` 속성을 통해 접근할 수 있습니다.

In [13]:
model.layers

`add()` 메서드를 통해 Sequential 모델을 점진적으로 작성할 수도 있습니다.

In [14]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu")) #layer를 이후에 추가할 수도 있음
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

레이어를 제거하는 `pop()` 메서드도 있습니다. Sequential 모델은 레이어의 리스트와 매우 유사하게 동작합니다.

In [15]:
model.pop()
print(len(model.layers))  

2


또한 Sequential 생성자는 Keras의 모든 레이어 또는 모델과 마찬가지로 `name` 인수를 허용합니다. 이것은 의미론적으로 유의미한 이름으로 TensorBoard 그래프에 주석을 달 때 유용합니다.

In [16]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

### 미리 입력 형상 지정하기

일반적으로 Keras의 모든 레이어는 가중치를 만들려면 입력의 형상을 알아야 합니다. 따라서 다음과 같은 레이어를 만들면 처음에는 가중치가 없습니다.

In [19]:
layer = layers.Dense(3)
layer.weights

[]

가중치는 모양이 입력의 형상에 따라 달라지기 때문에 입력에서 처음 호출될 때 가중치를 만듭니다.

In [20]:
x = tf.ones((1, 4))
y = layer(x)
layer.weights  

[<tf.Variable 'dense_8/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.03921849, -0.92059934, -0.57831067],
        [ 0.10424924,  0.51020956, -0.8262407 ],
        [-0.29044718,  0.80951273,  0.1645801 ],
        [-0.5329691 ,  0.79983604,  0.2861731 ]], dtype=float32)>,
 <tf.Variable 'dense_8/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

당연히 이것은 Sequential 모델에도 적용됩니다. 입력 형상이 없는 Sequential 모델을 인스턴스화할 때는 "빌드"되지 않습니다. 가중치가 없습니다(그리고 `model.weights`를 호출하면 오류가 발생함). 모델에 처음 입력 데이터가 표시되면 가중치가 생성됩니다.

In [22]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"), #(1,4)(4,2) = (1,2)
        layers.Dense(3, activation="relu"), #(1,2)(2,3 = (1,3))
        layers.Dense(4),                    #(1,3)(3,4) = (1,4)
    ]
)  

x = tf.ones((1, 4))
y = model(x)
print(model.weights)
print("Number of weights after calling the model:", len(model.weights))  # 6

[<tf.Variable 'dense_12/kernel:0' shape=(4, 2) dtype=float32, numpy=
array([[-8.6328173e-01, -9.2067456e-01],
       [-1.4947653e-02, -8.8691711e-05],
       [ 8.9718437e-01, -4.0868521e-02],
       [-6.9934249e-01, -2.0029974e-01]], dtype=float32)>, <tf.Variable 'dense_12/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, <tf.Variable 'dense_13/kernel:0' shape=(2, 3) dtype=float32, numpy=
array([[-0.9670933 , -1.0155296 ,  0.03211248],
       [ 0.36173737,  0.94941545, -0.91425586]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>, <tf.Variable 'dense_14/kernel:0' shape=(3, 4) dtype=float32, numpy=
array([[ 0.8878567 , -0.90674067,  0.09550548, -0.82324696],
       [ 0.46244228,  0.4841107 , -0.44089043,  0.7482898 ],
       [ 0.00936681,  0.15715003,  0.45346177, -0.17437685]],
      dtype=float32)>, <tf.Variable 'dense_14/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=flo

모델이 "빌드"되면, 그 내용을 표시하기 위해 `summary()` 메서드를 호출할 수 있습니다.

In [23]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (1, 2)                    10        
_________________________________________________________________
dense_13 (Dense)             (1, 3)                    9         
_________________________________________________________________
dense_14 (Dense)             (1, 4)                    16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


그러나 현재 출력 형상을 포함하여 지금까지 모델의 요약을 표시할 수 있도록 Sequential 모델을 점진적으로 빌드할 때 매우 유용할 수 있습니다. 이 경우 `Input` 객체를 모델에 전달하여 모델의 시작 형상을 알 수 있도록 모델을 시작해야 합니다.

In [25]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))               #(?,4)
model.add(layers.Dense(2, activation="relu"))    #(?,4)(4,2) => (?,2) param=10

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


`Input` 객체는 레이어가 아니므로 `model.layers`의 일부로 표시되지 않습니다.

In [16]:
model.layers

간단한 대안은 첫 번째 레이어에 `input_shape` 인수를 전달하는 것입니다.

In [26]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,))) # (None,4)(4,2) = (None,2) param=10

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


이처럼 사전 정의된 입력 모양으로 빌드된 모델은 항상 가중치를 가지며(데이터를 보기 전에도) 항상 정의된 출력 형상을 갖습니다.

일반적으로 Sequential 모델의 입력 형상을 알고 있는 경우 항상 Sequential 모델의 입력 형상을 지정하는 것이 좋습니다.